<br>
<br>

# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Modelos seq2seq 2**

Cuando la entrada de cada paso del decodificador proviene de la salida del paso anterior, estamos hablando de un modelo Seq2Seq con realimentación (feedback). Esto es especialmente común en tareas como la generación de texto.

1. **Inicio de la Secuencia**: Se inicia la generación con un token especial, como `<SOS>` (Start of Sequence).

2. **Generación Paso a Paso**:
   - En el primer paso, el decodificador recibe el `<SOS>` y el estado del codificador como entrada.
   - El decodificador procesa esta entrada y genera una salida para este paso.
   - La salida generada se convierte en la entrada para el siguiente paso, junto con el estado actualizado del decodificador.
   - Este proceso se repite hasta que se genera un token especial de fin de secuencia (`<EOS>`, End of Sequence) o hasta alcanzar un límite máximo de longitud.

3. **Ventajas y Desventajas**:
   - **Ventajas**: Esta forma de generar secuencias puede ayudar a mantener la coherencia en las secuencias generadas, ya que cada nueva palabra o token tiene en cuenta lo que ya se ha generado.
   - **Desventajas**: Puede ser más lento, ya que cada paso depende del anterior, y errores en un paso pueden propagarse y afectar los pasos siguientes.


Imagina que tienes un modelo seq2seq entrenado para traducir inglés a español. Para la frase "How are you?", el proceso sería algo así:

1. El codificador procesa "How are you?" y genera un vector latente.
2. El decodificador recibe el vector latente y el token `<SOS>`.
3. El decodificador genera "¿Cómo", actualiza su estado y toma "¿Cómo" como entrada para el siguiente paso.
4. El decodificador genera "estás", actualiza su estado y toma "estás" como entrada para el siguiente paso.
5. Y así sucesivamente, hasta generar `<EOS>` para indicar el final de la secuencia.

Este modelo de generación permite que el decodificador tenga en cuenta no solo el contexto proporcionado por el codificador, sino también la estructura de la secuencia que está generando, paso a paso.


<p align="center">
<img src="imgs/seq2seq_feedback.svg" width="70%">
</p>


### **Teacher Forcing**

**Teacher forcing** es una técnica utilizada en el entrenamiento de modelos seq2seq en la que, en un porcentaje de las veces, se utiliza la salida real (etiqueta) de un paso de tiempo como entrada para el siguiente paso, en lugar de la salida predicha por el modelo. Esta técnica puede ayudar a acelerar la convergencia y mejorar el rendimiento del modelo, especialmente en las etapas iniciales del entrenamiento.

#### **¿Cómo funciona?**

1. **Durante el Entrenamiento**: En cada paso de tiempo y con una cierta probabilidad de que suceda, en lugar de pasar la predicción del modelo del paso anterior como entrada al siguiente paso, se pasa la palabra real de la secuencia objetivo. Esto proporciona al modelo información directa y clara sobre cómo debería haber respondido en el paso anterior, independientemente de si la predicción fue correcta o no.

2. **Durante la Evaluación/Predicción**: El modelo debe generar secuencias por sí mismo, utilizando sus propias predicciones del paso anterior para el siguiente paso. Durante esta fase, no se utiliza "teacher forcing".

#### **Ventajas de Teacher Forcing:**

1. **Aprendizaje más Rápido**: Al proporcionar al modelo la respuesta correcta en cada paso, se reduce la propagación de errores a través de la secuencia, lo que puede llevar a un aprendizaje más rápido.

2. **Mejor Rendimiento**: Puede resultar en un mejor rendimiento del modelo, especialmente en las primeras etapas del entrenamiento.

### **Implementación traductor inglés a español**

#### **Dataset Europarl**

El conjunto de datos Europarl contiene las transcripciones de los procedimientos del Parlamento Europeo, proporcionando una valiosa fuente de textos paralelos en 21 idiomas europeos. Las oraciones están alineadas entre los idiomas, lo que lo hace especialmente útil para tareas de traducción automática.

Descargamos el dataset Europarl para español-inglés. Una vez descargado tendremos dos ficheros de texto, uno para cada idioma con las frases alineadas. El código siguiente muestra las primeras frases de cada fichero.

In [ ]:
def read_translation(archivo_ingles, archivo_espanol):
    with open(archivo_ingles, 'r', encoding='utf-8') as f_ingles, open(archivo_espanol, 'r', encoding='utf-8') as f_espanol:
        for oracion_ingles, oracion_espanol in zip(f_ingles, f_espanol):
            yield oracion_ingles.strip(), oracion_espanol.strip()


archivo_ingles = 'data/europarl/europarl-v7.es-en.en'
archivo_espanol = 'data/europarl/europarl-v7.es-en.es'

# Leer el conjunto de datos
for i, (ingles, espanol) in enumerate(read_translation(archivo_ingles, archivo_espanol)):
    print('Inglés:', ingles)
    print('Español:', espanol)
    print('---')
    if i == 15:
        break

FileNotFoundError: [Errno 2] No such file or directory: 'data/europarl/europarl-v7.es-en.en'

In [2]:
#! pip uninstall torch torchtext -y
!pip install torch==2.0.1 torchtext==0.15.2
!pip install portalocker>=2.0.0

ERROR: Could not find a version that satisfies the requirement torch==2.0.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==2.0.1
zsh:1: 2.0.0 not found


In [ ]:
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import torchtext
import torch
from collections import defaultdict


class Translation(Dataset):
    def __init__(self, source_file, target_file):
        self.ingles = []
        self.espanol = []
        self.tokenizer_es = get_tokenizer("spacy", language="es_core_news_md")
        self.tokenizer_en = get_tokenizer("spacy", language="en_core_web_md")
        self.vocab_es = torchtext.vocab.FastText(language='es', unk_init=torch.Tensor.normal_)  # <-- Mirar esto para ver si añadir el token <unk> al vocabulario
        self.vocab_en = torchtext.vocab.FastText(language='en', unk_init=torch.Tensor.normal_)

        self.vocab_en = self.add_sos_eos_unk_pad(self.vocab_en)
        self.vocab_es = self.add_sos_eos_unk_pad(self.vocab_es)

        self.archivo_ingles = source_file
        self.archivo_espanol = target_file

        # Leer el conjunto de datos
        for ingles, espanol in self.read_translation():
            self.ingles.append(ingles)
            self.espanol.append(espanol)


    def add_sos_eos_unk_pad(self, vocabulary):
        words = vocabulary.itos
        vocab = vocabulary.stoi
        embedding_matrix = vocabulary.vectors

        # Tokens especiales
        sos_token = '<sos>'
        eos_token = '<eos>'
        pad_token = '<pad>'
        unk_token = '<unk>'

        # Inicializamos los vectores para los tokens especiales, por ejemplo, con ceros
        sos_vector = torch.full((1, embedding_matrix.shape[1]), 1.)
        eos_vector = torch.full((1, embedding_matrix.shape[1]), 2.)
        pad_vector = torch.zeros((1, embedding_matrix.shape[1]))
        unk_vector = torch.full((1, embedding_matrix.shape[1]), 3.)

        # Añade los vectores al final de la matriz de embeddings
        embedding_matrix = torch.cat((embedding_matrix, sos_vector, eos_vector, unk_vector, pad_vector), 0)

        # Añade los tokens especiales al vocabulario
        vocab[sos_token] = len(vocab)
        vocab[eos_token] = len(vocab)
        vocab[pad_token] = len(vocab)
        vocab[unk_token] = len(vocab)

        words.append(sos_token)
        words.append(eos_token)
        words.append(pad_token)
        words.append(unk_token)

        vocabulary.itos = words
        vocabulary.stoi = vocab
        vocabulary.vectors = embedding_matrix

        default_stoi = defaultdict(lambda : len(vocabulary)-1, vocabulary.stoi)
        vocabulary.stoi = default_stoi

        return vocabulary


    def read_translation(self):
        with open(self.archivo_ingles, 'r', encoding='utf-8') as f_ingles, open(self.archivo_espanol, 'r', encoding='utf-8') as f_espanol:
            for oracion_ingles, oracion_espanol in zip(f_ingles, f_espanol):
                yield oracion_ingles.strip().lower(), oracion_espanol.strip().lower()

    def __len__(self):
        return len(self.ingles)

    def __getitem__(self, idx):
        item = self.ingles[idx], self.espanol[idx]
        tokens_ingles = self.tokenizer_en(item[0])
        tokens_espanol = self.tokenizer_es(item[1])

        tokens_ingles = tokens_ingles + ['<eos>']
        tokens_espanol = ['<sos>'] + tokens_espanol + ['<eos>']

        if not tokens_ingles or not tokens_espanol:
            return torch.zeros(1, 300), torch.zeros(1, 300)
            # raise RuntimeError("Una de las muestras está vacía.")

        tensor_ingles = self.vocab_en.get_vecs_by_tokens(tokens_ingles)
        tensor_espanol = self.vocab_es.get_vecs_by_tokens(tokens_espanol)

        indices_ingles = [self.vocab_en.stoi[token] for token in tokens_ingles] + [self.vocab_en.stoi['<pad>']]
        indices_espanol = [self.vocab_es.stoi[token] for token in tokens_espanol] + [self.vocab_es.stoi['<pad>']]

        return tensor_ingles, tensor_espanol, indices_ingles, indices_espanol



def collate_fn(batch):
    ingles_batch, espanol_batch, ingles_seqs, espanol_seqs = zip(*batch)
    ingles_batch = pad_sequence(ingles_batch, batch_first=True, padding_value=0)
    espanol_batch = pad_sequence(espanol_batch, batch_first=True, padding_value=0)

    # Calcular la longitud máxima de la lista de listas de índices
    pad = espanol_seqs[0][-1]  # token <pad>
    max_len = max([len(l) for l in espanol_seqs])
    for seq in espanol_seqs:
        seq += [pad]*(max_len-len(seq))

    return ingles_batch, espanol_batch, ingles_seqs, espanol_seqs

In [ ]:
!python -m spacy download es_core_news_md
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 31.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 23.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# archivo_ingles = 'data/europarl/europarl-v7.es-en.en'
# archivo_espanol = 'data/europarl/europarl-v7.es-en.es'

archivo_ingles = 'mock.en'
archivo_espanol = 'mock.es'

translation = Translation(archivo_ingles, archivo_espanol)

.vector_cache/wiki.es.vec: 2.59GB [00:08, 305MB/s]                            
100%|██████████| 985667/985667 [01:50<00:00, 8913.58it/s]
.vector_cache/wiki.en.vec: 6.60GB [01:19, 82.9MB/s]                            
100%|██████████| 2519370/2519370 [04:43<00:00, 8892.07it/s]


### **Modelo**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

##### **Encoder**

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, x):
        output, (hidden, cell) = self.rnn(x)
        return output, (hidden, cell)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    """
    Implementación de la atención de Luong (tipo 'general').
    """
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        # Matriz de proyección para la variante "general"
        self.W = nn.Linear(hidden_dim, hidden_dim, bias=False)

    def forward(self, encoder_outputs, decoder_hidden):
        """
        Args:
            encoder_outputs (Tensor): Salidas del encoder,
                dimensión (batch_size, seq_len, hidden_dim).
            decoder_hidden (Tensor): Último estado oculto del decodificador
                o estado actual, dimensión (batch_size, hidden_dim).

        Returns:
            context_vector (Tensor): Contexto resultante de la atención,
                dimensión (batch_size, hidden_dim).
            attn_weights (Tensor): Pesos de atención por cada paso temporal
                de la secuencia de entrada, dimensión (batch_size, seq_len).
        """

        # 1) Proyectamos el estado oculto del decodificador (batch_size, hidden_dim)
        decoder_hidden_proj = self.W(decoder_hidden)  # (batch_size, hidden_dim)

        # 2) Calculamos los puntajes de atención usando producto punto:
        #    encoder_outputs:       (batch_size, seq_len, hidden_dim)
        #    decoder_hidden_proj:   (batch_size, hidden_dim) -> (batch_size, hidden_dim, 1)
        # => scores:               (batch_size, seq_len)
        scores = torch.bmm(encoder_outputs, decoder_hidden_proj.unsqueeze(2)).squeeze(2)

        # 3) Calculamos los pesos de atención aplicando softmax a los puntajes
        attn_weights = F.softmax(scores, dim=1)  # (batch_size, seq_len)

        # 4) Multiplicamos los pesos de atención por las salidas del encoder
        #    para obtener el vector de contexto
        #    attn_weights:   (batch_size, seq_len)
        #    encoder_outputs:(batch_size, seq_len, hidden_dim)
        # => context_vector:(batch_size, hidden_dim)
        context_vector = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)

        return context_vector, attn_weights


##### **Decoder**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.5):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Capa LSTM, recibe como entrada (input_dim + hidden_dim)
        self.rnn = nn.LSTM(input_dim + hidden_dim,
                           hidden_dim,
                           num_layers,
                           batch_first=True,
                           dropout=dropout)

        # Proyección final para obtener logits sobre el vocabulario
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        # Mecanismo de atención (Luong general)
        self.attention = Attention(hidden_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, hidden, cell, encoder_outputs):
        """
        Args:
            x (Tensor): Embedding del token actual -> (batch_size, 1, input_dim)
            hidden (Tensor): Estado oculto actual del LSTM -> (num_layers, batch_size, hidden_dim)
            cell (Tensor): Estado de celda actual del LSTM -> (num_layers, batch_size, hidden_dim)
            encoder_outputs (Tensor): Salidas del encoder -> (batch_size, seq_len, hidden_dim)

        Returns:
            output (Tensor): Logits proyectados -> (batch_size, 1, output_dim)
            (hidden, cell) (tuple): Nuevos estados oculto y de celda del LSTM
            attn_weights (Tensor): Pesos de atención -> (batch_size, seq_len)
        """
        # 1) Extraemos el estado oculto de la última capa (para el mecanismo de atención)
        dec_hidden = hidden[-1]  # (batch_size, hidden_dim)

        # 2) Calculamos contexto y pesos de atención con la clase 'Attention'
        context_vector, attn_weights = self.attention(encoder_outputs, dec_hidden)
        context_vector = context_vector.unsqueeze(1)  # (batch_size, 1, hidden_dim)

        # 3) Concatenar la entrada actual (x) con el contexto
        rnn_input = torch.cat((x, context_vector), dim=2)  # (batch_size, 1, input_dim + hidden_dim)
        rnn_input = self.dropout(rnn_input)

        # 4) Pasar por la LSTM
        output, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))

        # 5) Proyectamos para obtener la distribución de probabilidad sobre el vocab.
        prediction = self.fc_out(output)  # (batch_size, 1, output_dim)

        return prediction, (hidden, cell), attn_weights


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.es_embeddings = torchtext.vocab.FastText(language='es')
        self.M = self.es_embeddings.vectors
        self.M = torch.cat((self.M, torch.zeros((4, self.M.shape[1]))), 0)

    def forward(self, source, target, encoder_outputs=None, teacher_forcing_ratio=0.5):
        target_len = target.shape[1]
        batch_size = target.shape[0]

        # Tensor para almacenar las salidas del decoder
        outputs = torch.zeros(batch_size, target_len, 985671)

        # Si encoder_outputs no está definido, procesa la fuente con el encoder
        if encoder_outputs is None:
            encoder_outputs, (hidden, cell) = self.encoder(source)
        else:
            # Si encoder_outputs está definido, no procesamos source nuevamente
            hidden, cell = None, None  # O puedes inicializar con otros valores si es necesario

        # La primera entrada al decoder es el vector <sos>
        x = target[:, 0, :]

        for t in range(1, target_len):
            # Modification: Added '_' to ignore attention weights
            output, (hidden, cell), _ = self.decoder(x.unsqueeze(1), hidden, cell, encoder_outputs)
            outputs[:, t, :] = output.squeeze(1)

            teacher_force = random.random() < teacher_forcing_ratio
            if teacher_force:
                x = target[:, t, :]
            else:
                x = torch.matmul(output.squeeze(1), self.M)

        return outputs

In [ ]:
# Parámetros
input_dim = 300
output_dim = translation.vocab_es.vectors.shape[0]
hidden_dim = 512
num_layers = 2
learning_rate = 0.001
num_epochs = 30
batch_size = 8
num_workers = 0
shuffle = True

# Inicializa el modelo, el optimizador y la función de pérdida
encoder = Encoder(input_dim, hidden_dim, num_layers)
decoder = Decoder(input_dim, hidden_dim, output_dim, num_layers)
model = Seq2Seq(encoder, decoder)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# DataLoader
from torch.utils.data import DataLoader

dataloader = DataLoader(translation, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=collate_fn)

In [ ]:
# Bucle de entrenamiento
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, (src, tgt, src_indices, tgt_indices) in enumerate(dataloader):
        optimizer.zero_grad()
        output = model(src, tgt)

        tgt_indices = torch.tensor(tgt_indices, dtype=torch.long)
        loss = 0
        for t in range(1, tgt.shape[1]):
            loss += criterion(output[:, t, :], tgt_indices[:, t])
        # loss = criterion(output, torch.tensor(tgt_indices, dtype=torch.long))

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if batch_idx % 5 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss / len(dataloader):.4f}')

Epoch [1/30], Step [1/2], Loss: 41.4527
Epoch [1/30], Average Loss: 41.3835
Epoch [2/30], Step [1/2], Loss: 40.8925
Epoch [2/30], Average Loss: 33.8316
Epoch [3/30], Step [1/2], Loss: 39.0843
Epoch [3/30], Average Loss: 38.0465
Epoch [4/30], Step [1/2], Loss: 34.2481
Epoch [4/30], Average Loss: 27.4220
Epoch [5/30], Step [1/2], Loss: 27.6008
Epoch [5/30], Average Loss: 26.6998
Epoch [6/30], Step [1/2], Loss: 22.8337
Epoch [6/30], Average Loss: 22.1145
Epoch [7/30], Step [1/2], Loss: 18.9444
Epoch [7/30], Average Loss: 19.5379
Epoch [8/30], Step [1/2], Loss: 16.5222
Epoch [8/30], Average Loss: 16.2344
Epoch [9/30], Step [1/2], Loss: 14.0096
Epoch [9/30], Average Loss: 14.5125
Epoch [10/30], Step [1/2], Loss: 11.7373
Epoch [10/30], Average Loss: 12.7249
Epoch [11/30], Step [1/2], Loss: 10.1369
Epoch [11/30], Average Loss: 10.5326
Epoch [12/30], Step [1/2], Loss: 9.4058
Epoch [12/30], Average Loss: 7.8522
Epoch [13/30], Step [1/2], Loss: 7.7582
Epoch [13/30], Average Loss: 7.7318
Epoch [1

In [ ]:
# Test the model with input sentences
model.eval()

sentence = "my cat"

# Convertir a vectores
tokens = translation.tokenizer_en(sentence)
tokens = tokens + ['<eos>']
text_tensor = translation.vocab_en.get_vecs_by_tokens(tokens)
text_tensor = text_tensor.unsqueeze(0)

with torch.no_grad():
    encoder_outputs, (hidden, cell) = model.encoder(text_tensor)

outputs = []

input_token = torch.tensor(translation.vocab_es.stoi['<sos>']).unsqueeze(0)
input_token = translation.vocab_es.vectors[input_token].unsqueeze(0)


for _ in range(5):
    with torch.no_grad():
        # Modification: Added '_' to ignore attention weights
        output, (hidden, cell), _ = model.decoder(input_token, hidden, cell, encoder_outputs) # teacher_forcing_ratio=0.0

    # Obtener el token con la probabilidad más alta
    best_guess = output.argmax(2).squeeze(0)
    outputs.append(best_guess.item())

    # Si el token es <eos>, terminar la traducción
    if best_guess == translation.vocab_es.stoi['<eos>']:
        break

    # Utilizar la palabra predicha como la siguiente entrada al decoder
    input_token = translation.vocab_es.vectors[best_guess].unsqueeze(0)

# Convertir los índices de salida a palabras
translated_sentence = [translation.vocab_es.itos[idx] for idx in outputs]

result = ' '.join(translated_sentence)

print(result)

mi <eos>


In [ ]:
# Guardar el modelo
torch.save(model.state_dict(), 'seq2seq.pth')

In [ ]:
# Load model from file
model.load_state_dict(torch.load('seq2seq.pth'))

<All keys matched successfully>